### Importing Necessary Libraries

In [8]:
import minsearch
import json

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

### Document preparation

In [9]:
with open('./documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Implemenation of Required Functions

In [10]:
def execute_search(index, query, filter_options, boost_options, num_results):
    """Performs a search on the given index with specified filters and boosting."""
    return index.search(
        query=query,
        filter_dict=filter_options,
        boost_dict=boost_options,
        num_results=num_results
    )

def format_prompt(search_results, question, context_template):
    """Formats a prompt using the search results, question, and a context template."""
    formatted_context = ""

    for result in search_results:
        formatted_context += f"section: {result['section']}\nquestion: {result['question']}\ntext: {result['text']}\n\n"

    return context_template.format(question=question, context=formatted_context)

def ask_question_with_context(model, question, context):
    """Uses a model to generate an answer based on a question and a context."""
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()
    client = OpenAI()
    prompt = format_prompt(context, question, prompt_template)
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

# Example of how these functions could be used together:
def rag(model, index, question, filter_dict, boost_dict, num_results):
    search_results = execute_search(index, question, filter_dict, boost_dict, num_results)
    answer = ask_question_with_context(model, question, search_results)
    return answer

### Indexing Data via Toy Search Engine

In [11]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)
index.fit(documents)

### Performing RAG

In [12]:
model = 'gpt-4o'
filter_dict = {'course': 'data-engineering-zoomcamp'}
boost_dict = {'question': 3.0, 'section': 0.5}
num_results = 5

question = 'The course has already started. Can I still enroll?'

In [13]:
rag(model, index, question, filter_dict, boost_dict, num_results)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### Indexing Data using Elasticsearch

In [14]:
from elasticsearch import Elasticsearch

In [15]:
es_client = Elasticsearch('http://localhost:9200')

In [17]:
print(es_client.info())

{'name': '69b01e627377', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gZzlOp3LQlSdZJeRd6BIOg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [18]:
from tqdm.auto import tqdm

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
from tqdm.auto import tqdm

In [24]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [25]:
question

'The course has already started. Can I still enroll?'

In [37]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [38]:
elastic_search(question)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [39]:
# Example of how these functions could be used together:
def rag(model, question):
    search_results = elastic_search(question)
    answer = ask_question_with_context(model, question, search_results)
    return answer

In [41]:
rag(model, question)

"Yes, you are still eligible to submit homework even if you don't register. However, be aware of the deadlines for turning in the final projects and avoid leaving everything for the last minute."